# Table joining
Joining tables is part of the daily life of data scientists. In this notebook you will learn how joins and unions work. We start by importing the required pyspark machinery and some other functions we will use to create the tables.

In [2]:
# Spark related machinery
import pyspark
import pyspark.sql.functions as F
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql.functions import concat_ws

spark = pyspark.sql.SparkSession.builder.enableHiveSupport().getOrCreate()

In [3]:
from pyspark_functions import create_players_table,\
                              create_new_players_table,\
                              create_ranking_table

## Table creation

We start by creating some tables, in this case we will have two tables with the information of video game players, and a second table with their rankings:

* **players**: Contains the name of the player, how old the player is, the game he/she plays, and the player id
* **new_players**: this table is a continuation of the table players, it has the same columns 
* **ranking**: Contains the player’s id, and the player’s ranking

Below we create and use the method ```show()``` to print the tables:

In [4]:
players = create_players_table()
new_players = create_new_players_table()
ranking = create_ranking_table()

players.show()
new_players.show()
ranking.show()

+-------+---+--------------+---------+
|   Name|Age|          Game|Player_id|
+-------+---+--------------+---------+
|Nicolas| 25|          Doom|        1|
| Camila| 23|        Diablo|        2|
|Gabriel| 20|   Wolfenstein|        3|
|  Mateo| 23|    Zelda BOTW|        4|
|   Luna| 21|         Mario|        5|
|   Lily| 25|Counter Strike|        6|
|  Sofia| 26|     Max Payne|        7|
|    Leo| 28|       Fifa 20|        8|
| Thomas| 22|         Speed|        9|
|  James| 30| Goldeneye 007|       15|
+-------+---+--------------+---------+

+-----+---+------+---------+
| Name|Age|  Game|Player_id|
+-----+---+------+---------+
|  Mia| 22| Mario|       10|
|David| 28|Diablo|       11|
|Dylan| 21|  Doom|       12|
+-----+---+------+---------+

+---------+-------+
|Player_id|Ranking|
+---------+-------+
|        1|      1|
|        2|      1|
|        3|      4|
|        4|      9|
|        5|      2|
|        6|      3|
|        7|     99|
|        8|     22|
|        9|     12|
|       1

Ok, we have the tables ready. By now you should have notice that the new_players table is just a continuation of the players table and also that the column Player_id is common to all 3 tables. Therefore, we will be joining the tables **ranking** and **players**, while we will union **players** and **new_players**.

# Inner join example
In the inner join only the elements common to both tables are preserved. As an example we will be joining the tables **players** and **ranking**:

In [5]:
#The join
players_ranking_inner = players.join(ranking, on="Player_id", how="inner")

#sort the table by player_id and print it
players_ranking_inner.sort("Player_id").show()

+---------+-------+---+--------------+-------+
|Player_id|   Name|Age|          Game|Ranking|
+---------+-------+---+--------------+-------+
|        1|Nicolas| 25|          Doom|      1|
|        2| Camila| 23|        Diablo|      1|
|        3|Gabriel| 20|   Wolfenstein|      4|
|        4|  Mateo| 23|    Zelda BOTW|      9|
|        5|   Luna| 21|         Mario|      2|
|        6|   Lily| 25|Counter Strike|      3|
|        7|  Sofia| 26|     Max Payne|     99|
|        8|    Leo| 28|       Fifa 20|     22|
|        9| Thomas| 22|         Speed|     12|
+---------+-------+---+--------------+-------+



As you can see we are missing the rows where player_id = 10, 11, 15.

# Left join example

In the left join only the elements of the left table (the first table) are preserved, and for the cases where the joining key is not found in the right table, the new columns (the ones in comming from the right table) are filled with null values.

In [7]:
#The join
ranking_players_left = ranking.join(players, on="Player_id", how="left")

#Sort the table by player_id and print it
ranking_players_left.sort("Player_id").show()

+---------+-------+-------+----+--------------+
|Player_id|Ranking|   Name| Age|          Game|
+---------+-------+-------+----+--------------+
|        1|      1|Nicolas|  25|          Doom|
|        2|      1| Camila|  23|        Diablo|
|        3|      4|Gabriel|  20|   Wolfenstein|
|        4|      9|  Mateo|  23|    Zelda BOTW|
|        5|      2|   Luna|  21|         Mario|
|        6|      3|   Lily|  25|Counter Strike|
|        7|     99|  Sofia|  26|     Max Payne|
|        8|     22|    Leo|  28|       Fifa 20|
|        9|     12| Thomas|  22|         Speed|
|       10|    440|   null|null|          null|
|       11|     21|   null|null|          null|
+---------+-------+-------+----+--------------+



As you can see, all the elements from the table ranking were preserved. causing the last two rows to be fill with **null** values for the column **Name**, **Age**, and **Game**.

# Outer join Example
In the outer join all the keys in both tables are preseved. This means that the columns for which the joining key is not in both tables are filled with null values. In the following block of code we show an example:

In [6]:
#The join
ranking_players_outer = ranking.join(players, on="Player_id", how="outer")

#Sort the table by player_id and print it
ranking_players_outer.sort("Player_id").show()

+---------+-------+-------+----+--------------+
|Player_id|Ranking|   Name| Age|          Game|
+---------+-------+-------+----+--------------+
|        1|      1|Nicolas|  25|          Doom|
|        2|      1| Camila|  23|        Diablo|
|        3|      4|Gabriel|  20|   Wolfenstein|
|        4|      9|  Mateo|  23|    Zelda BOTW|
|        5|      2|   Luna|  21|         Mario|
|        6|      3|   Lily|  25|Counter Strike|
|        7|     99|  Sofia|  26|     Max Payne|
|        8|     22|    Leo|  28|       Fifa 20|
|        9|     12| Thomas|  22|         Speed|
|       10|    440|   null|null|          null|
|       11|     21|   null|null|          null|
|       15|   null|  James|  30| Goldeneye 007|
+---------+-------+-------+----+--------------+



As you can see only the rows where player_id = 10, 11, and 15 contain null values in the some of the columns (these player ids are not commun to both tables).

# Left anti join example
In a left anti join only the keys of the left table (the first table) that are not present in the second table are preserved:

In [8]:
#The join
ranking_players_left_anti = players.join(ranking, on="Player_id", how="left_anti")

#Print the table
ranking_players_left_anti.show()

+---------+-----+---+-------------+
|Player_id| Name|Age|         Game|
+---------+-----+---+-------------+
|       15|James| 30|Goldeneye 007|
+---------+-----+---+-------------+



As you can see we end up with only one row. Player_id = 15 is in the table **players** but not in the table **ranking**.

# Union tables
Sometimes you have a dataset split into two tables, as in the case of the **players** and **new_players** tables, and you want to "union" them (put one after the other). This is done as follows:

In [10]:
#List with the name of the columns
cols = players.columns

#Union
full_players = players.union(new_players.select(cols))

#Print to screen the new table
full_players.sort("Player_id").show()

+-------+---+--------------+---------+
|   Name|Age|          Game|Player_id|
+-------+---+--------------+---------+
|Nicolas| 25|          Doom|        1|
| Camila| 23|        Diablo|        2|
|Gabriel| 20|   Wolfenstein|        3|
|  Mateo| 23|    Zelda BOTW|        4|
|   Luna| 21|         Mario|        5|
|   Lily| 25|Counter Strike|        6|
|  Sofia| 26|     Max Payne|        7|
|    Leo| 28|       Fifa 20|        8|
| Thomas| 22|         Speed|        9|
|    Mia| 22|         Mario|       10|
|  David| 28|        Diablo|       11|
|  Dylan| 21|          Doom|       12|
|  James| 30| Goldeneye 007|       15|
+-------+---+--------------+---------+



In the first line we are creating a list with the name of the columns in the table players, while the second line is where the union takes place. It is important to notice that we select the columns of the table **players** in the table **new_players** (second line). The reason for this is to ensure that the order of the columns is the same in both tables when the union takes place. If the columns are not in the same order on both tables, the union won't error and you will end up having a table with the columns mixed (see below). While this is very easy to spot in a small table, it is not so easy to spot when you have a couple million rows.

In [12]:
players.union(new_players.select(["Name", "Age", "Player_id", "Game"])).show()

+-------+---+--------------+---------+
|   Name|Age|          Game|Player_id|
+-------+---+--------------+---------+
|Nicolas| 25|          Doom|        1|
| Camila| 23|        Diablo|        2|
|Gabriel| 20|   Wolfenstein|        3|
|  Mateo| 23|    Zelda BOTW|        4|
|   Luna| 21|         Mario|        5|
|   Lily| 25|Counter Strike|        6|
|  Sofia| 26|     Max Payne|        7|
|    Leo| 28|       Fifa 20|        8|
| Thomas| 22|         Speed|        9|
|  James| 30| Goldeneye 007|       15|
|    Mia| 22|            10|    Mario|
|  David| 28|            11|   Diablo|
|  Dylan| 21|            12|     Doom|
+-------+---+--------------+---------+



## Final words

Ok, now you know the basic tools to join and union tables. Now is your time to start coding!. You can start by changing the order of the tables in the left join.